In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '1' # speed up
import numpy as np
from MMO.problems.common import build_problem
from MMO.mobo.algorithms import get_algorithm
from MMO.visualization.data_export import DataExport
from MMO.arguments import get_args
from MMO.utils import save_args, setup_logger
import networkx as nx
import random, pickle, string
import time
import os, sys
from random import randint, shuffle, sample

sys.setrecursionlimit(2000000)
MAX_COUNT_INDEX = 3001946 # 3001946 3001946
from pymoo.optimize import minimize
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.algorithms.moead import MOEAD

import matplotlib.pyplot as pl
from  problem.network_optimization.topology import NetTopology

def main():
   
    # load arguments
    args, framework_args = get_args()
    
    # set seed
    np.random.seed(args.seed)
    tempCacheInfo = {'Sydney1': {'size': 100, 'type': 'LRU'}, 'Brisbane2': {'size': 100, 'type': 'LRU'}, 'Canberra1': {'size': 100, 'type': 'LRU'}, \
                 'Sydney2': {'size': 100, 'type': 'LRU'}, 'Townsville': {'size': 100, 'type': 'LRU'}, 'Cairns': {'size': 100, 'type': 'LRU'}, \
                 'Brisbane1': {'size': 100, 'type': 'LRU'}, 'Rockhampton': {'size': 100, 'type': 'LRU'}, 'Armidale': {'size': 100, 'type': 'LRU'}, \
                 'Hobart': {'size': 100, 'type': 'LRU'}, 'Canberra2': {'size': 100, 'type': 'LRU'}, 'Perth1': {'size': 100, 'type': 'LRU'}, \
                 'Perth2': {'size': 100, 'type': 'LRU'}, 'Adelaide1': {'size': 100, 'type': 'LRU'}, 'Adelaide2': {'size': 100, 'type': 'LRU'}, \
                 'Melbourne1': {'size': 100, 'type': 'LRU'}, 'Melbourne2': {'size': 100, 'type': 'LRU'}, 'Alice Springs': {'size': 100, 'type': 'LRU'}, \
                 'Darwin': {'size': 100, 'type': 'LRU'}}
    bounds = [1, 100]
    topology = NetTopology('topology/Aarnet.gml', 'Sydney1', cacheDictInfo=tempCacheInfo)
    runReqNums = 1000
    # build problem, get initial samples
    extra_params = {'topo': topology, "runReqNums": runReqNums}
    
    problem, pareto_front, X_init, Y_init = build_problem(args.problem, args.n_var, args.n_obj, args.n_init_sample, args.n_process, extra_params=extra_params)
    
    args.n_var, args.n_obj = problem.n_var, problem.n_obj
    print("Start the optimizer")
    # initialize optimizer
    optimizer = get_algorithm(args.algo)(problem, args.n_iter, args.ref_point, framework_args)

    # save arguments & setup logger
    save_args(args, framework_args)
    logger = setup_logger(args)
    print(problem, optimizer, sep='\n')
    
    # initialize data exporter
    exporter = DataExport(optimizer, X_init, Y_init, args)

    # optimization
    solution = optimizer.solve(X_init, Y_init)

    # export true Pareto front to csv
#     if true_pfront is not None:
#         exporter.write_truefront_csv(true_pfront)

    for _ in range(args.n_iter):
        # get new design samples and corresponding performance
        X_next, Y_next, _ = next(solution)
        
        # update & export current status to csv
        exporter.update(X_next, Y_next)
        exporter.write_csvs()

    # close logger
    if logger is not None:
        logger.close()

main()


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-sya1vtkr because the default path (/home/picarib/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Start the optimizer
========== Problem Definition ==========
# name: CDN_PLACEMENT
# n_var: 19
# n_obj: 2
# n_constr: 0
# f(xl): 1.0
# f((xl+xu)/2): 1.0
# f(xu): 0.10619469026548672

========== Framework Description ==========
# algorithm: DGEMO
# surrogate: GaussianProcess
# acquisition: IdentityFunc
# solver: ParetoDiscoverySolver
# selection: DGEMOSelect

========== Iteration 0 ==========
Surrogate model fitted: 0.40s


/home/picarib/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__k2__length_scale is close to the specified upper bound 31.622776601683793. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/home/picarib/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__k2__length_scale is close to the specified upper bound 31.622776601683793. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/home/picarib/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 8 of parameter k1__k2__length_scale is close to the specified upper bound 31.622776601683793. Increasing the bound and calling fi

ParetoDiscovery optimizing: generation 3